# Go.com Application Exercise

## Assumptions
- Assume that go.com is a fictional publicly traded ecommerce company that sells
    goods across three websites: cars.go.com (which sells
    cars), boats.go.com (which sells boats), and planes.go.com (which sells planes).
- Assume it is April 2015 and Go.com is about to report their financial results for
    their most recent quarter (1Q15, ended on March 31, 2015).
- Assume that by collecting data from Go.com's three websites, we've been able to
    come up with very accurate estimates for what Go.com will report for 1q15
    revenue.
- The attached Excel workbook contains a) the daily revenue data that we’ve
    collected from Go.com’s three websites for 1Q15 (this is on the sales data tab,
    each row represents total revenue earned by a particular site for that day) and b)
    the company reported data for the last five quarters for both revenue and profit
    (historical data tab)
    
## Question
  - **Based on this 1q15 revenue data and the historical data, your assignment is to
     come up with an estimate for what Go.com will report as their 1q15 profit.** 
      -  In your response, please include both your 1q15 profit estimate and a short description of how you arrived at that estimate. Please note any data irregularities that you find.

In [1]:
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.filterwarnings("ignore")

### Preprocessing

In [2]:
go_xls = pd.ExcelFile('Go.com_Assignment_Data.xlsx')

# historical revenue data
RevenueHistData=pd.read_excel(go_xls,sheet_name=go_xls.sheet_names[1]).iloc[6:10, 1:].T
RevenueHistData.rename(columns=RevenueHistData.iloc[0],inplace=True)
RevenueHistData.drop(RevenueHistData.index[0],inplace=True)
RevenueHistData.set_index(pd.period_range('2013Q4','2014Q4',freq='Q'),'date',inplace=True)

# historical profit data
ProfitHistData=pd.read_excel(go_xls,sheet_name=go_xls.sheet_names[1]).iloc[13:17, 1:].T
ProfitHistData.rename(columns=ProfitHistData.iloc[0],inplace=True)
ProfitHistData.drop(ProfitHistData.index[0],inplace=True)
ProfitHistData.set_index(pd.period_range('2013Q4','2014Q4',freq='Q'),'date',inplace=True)

# Daily Sales data 
DailySalesData_2015Q1 = pd.read_excel(go_xls, sheet_name= go_xls.sheet_names[0])
DailySalesData_2015Q1 = DailySalesData_2015Q1.iloc[4:, 1:].reset_index().iloc[:, 1:]
DailySalesData_2015Q1.columns = ['date', 'revenue', 'product_line']
DailySalesData_2015Q1['date']= DailySalesData_2015Q1['date'].astype('datetime64[ns]')

### Preprocessed data

- **Historical Revenue Data ( 2013Q4 : 2014Q4 )**

In [3]:
RevenueHistData

,Cars.go.com,Planes.go.com,Boats.go.com,Total
2013Q4,6085063,32339403,4329578,42754044
2014Q1,5663582,34861524,4418088,44943194
2014Q2,5701994,36039564,4657184,46398742
2014Q3,5691910,34422242,4597765,44711917
2014Q4,5906352,45337642,5437214,56681208


- **Historical Profit Data ( 2013Q4 : 2014Q4 )**

In [4]:
ProfitHistData

,Cars.go.com,Planes.go.com,Boats.go.com,Total
2013Q4,608506,161697,86591.6,856795
2014Q1,566358,174308,88361.8,829028
2014Q2,570199,180198,93143.7,843541
2014Q3,569191,172111,91955.3,833258
2014Q4,590635,226688,108744,926068


- **Sales Data (Daily Records of Q1:2015)** 

In [5]:
DailySalesData_2015Q1.head(10)

,date,revenue,product_line
0,2015-01-01,108632,Cars.go.com
1,2015-01-01,31494,Planes.go.com
2,2015-01-01,239376,Boats.go.com
3,2015-01-02,552065,Cars.go.com
4,2015-01-02,238344,Planes.go.com
5,2015-01-02,37594.6,Boats.go.com
6,2015-01-03,334414,Cars.go.com
7,2015-01-03,51614,Planes.go.com
8,2015-01-03,24042.7,Boats.go.com
9,2015-01-04,320345,Cars.go.com


### Correlation between the Historical Revenues and Profits by website

In [6]:
# Division of each website's revenue by porift for each period
Divisions_RevenueByProfit=RevenueHistData.iloc[:,0:3]/ProfitHistData.iloc[:,0:3]
Divisions_RevenueByProfit

,Cars.go.com,Planes.go.com,Boats.go.com
2013Q4,10,200,50
2014Q1,10,200,50
2014Q2,10,200,50
2014Q3,10,200,50
2014Q4,10,200,50


* For the historical data, consistently,
  - **"Cars.go.com"**, Revenue is **10** times of Profit.
  - **"Planes.go.com"**, Revenue is **200** times of Profit.
  - **"Boats.go.com"**, Revenue is **50** times of Profit.

**Since the correlations between Revenues and Profits are historically consistent by the respective websites across 2013Q4 to 2014Q4, it is reasonable to make a prediction taking advantage of that fact.**

- Step 1: Get the Total Revenues of each website in 2015Q1 using "SalesData_2015Q1"

In [7]:
TotalRevenues_2015Q1=DailySalesData_2015Q1.groupby('product_line').sum()
TotalRevenues_2015Q1

,revenue
product_line,
Boats.go.com,4736347.68
Cars.go.com,16757524.80
Planes.go.com,32529742.56


- Step 2: Divide each website's total revenues by the "historical correlations with their profits" to get each website's profit estimation for 2015Q1, then sum the 3 estimated profits to get the total profit estimate. 

In [8]:
BoatsProfitEstimate=TotalRevenues_2015Q1.iloc[0][0]/50  # Boats.go.com: Revenue= 50 * Profit.
CarsProfitEstimate=TotalRevenues_2015Q1.iloc[1][0]/10   # Cars.go.com: Revenue= 10 * Profit.
PlanesProfitEstimate=TotalRevenues_2015Q1.iloc[2][0]/200 # Planes.go.com: Revenue= 200 * Profit.
TotalProfitEstimate_2015Q1= BoatsProfitEstimate+CarsProfitEstimate+PlanesProfitEstimate

In [9]:
ProfitEsimates_2015Q1=pd.DataFrame({'date':['2015Q1'],
                                  'Cars.go.com': [CarsProfitEstimate],
                                  'Planes.go.com': [PlanesProfitEstimate],
                                  'Boats.go.com':[BoatsProfitEstimate],
                                  'Total':[TotalProfitEstimate_2015Q1]}).\
                                  set_index('date')

## Profit Estimates for 2015Q1 (Answer for the Question)

In [10]:
ProfitEsimates_2015Q1

,Cars.go.com,Planes.go.com,Boats.go.com,Total
date,,,,
2015Q1,1675752.48,162648.7128,94726.9536,1.933128e+06


**Comparison with Historical Profit Data**

In [11]:
ProfitHistData

,Cars.go.com,Planes.go.com,Boats.go.com,Total
2013Q4,608506,161697,86591.6,856795
2014Q1,566358,174308,88361.8,829028
2014Q2,570199,180198,93143.7,843541
2014Q3,569191,172111,91955.3,833258
2014Q4,590635,226688,108744,926068


- **For "Planes.go.com" and "Boats.go.com", the Profit Estimates and their corresponding historical data are in the similar range.**

  **However, for "Cars.go.com," the Profit Esitmate is about 3 times of its corresponding historical data.**
     - **Due to this irregularity, I investigated if there are duplicate rows from "DailySalesData_2015Q1" where product line is "Cars.go.com"**

In [12]:
Cars_DailySalesData_2015Q1=DailySalesData_2015Q1[DailySalesData_2015Q1.product_line=='Cars.go.com']
DuplicateRows_df = Cars_DailySalesData_2015Q1[Cars_DailySalesData_2015Q1.duplicated(['revenue', 'date'])]
print("Number of Duplicate Rows based on 'revenue' and 'date' columns are:", len(DuplicateRows_df))

Number of Duplicate Rows based on 'revenue' and 'date' columns are: 0


- **There are no duplicate rows in "Cars_DailySalesData_2015Q1".**
  
  **Thus, the high profit estimate of "Cars.go.com" is attributed to the significantly higher revenue of "Cars.go.com" in 2015Q1**
  **compared to its history.**

- **A Simple Explanatory Data Analysis (EDA)"**
    - Investigated the monthly levels of revenue in 2015Q1 for "Cars.go.com," and see if there are any irregularities

In [13]:
Cars_DailySalesData_2015Q1.set_index('date', inplace=True);
Cars_DailySalesData_2015Q1['revenue']=Cars_DailySalesData_2015Q1['revenue'].astype('float64');
Cars_MonthlySalesData_2015Q1 = Cars_DailySalesData_2015Q1.resample('1M').sum()

In [14]:
Cars_MonthlySalesData_2015Q1 
RevenueHistData[['Cars.go.com']]

,revenue
date,
2015-01-31,5056747.2
2015-02-28,5329144.8
2015-03-31,6371632.8


,Cars.go.com
2013Q4,6085063
2014Q1,5663582
2014Q2,5701994
2014Q3,5691910
2014Q4,5906352


**Findings:**

- Cars.go.com's Monthly Sales (revenue) in 2015Q1 are in the similar range as its Quarterly Sales for the past 1 year and a quarter. 
- In 2015Q1, Cars.go.com's March monthly sales (revenue) increased even more than its previous 2 months.
